# EDA
Here, we'll conduct all of the EDA for this project.

In [12]:
import mysql.connector 
import config
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [14]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)
cursor = cnx.cursor()
cursor.execute("USE google_maps")

In [38]:
cursor.execute("""SELECT * FROM trips WHERE DAY(departure_time) > 18;""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df.head()

,departure_time,trip_direction,trip_duration
0,2020/02/22 10:05,0,66
1,2020/02/22 10:05,1,55
2,2020/02/22 10:10,0,66
3,2020/02/22 10:10,1,58
4,2020/02/22 10:15,0,55


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 3 columns):
departure_time    382 non-null object
trip_direction    382 non-null int64
trip_duration     382 non-null int64
dtypes: int64(2), object(1)
memory usage: 9.1+ KB


In [90]:
def get_trip_direction_text(value):
    if value == 0:
        return 'Trip to gf'
    elif value == 1:
        return 'Trip to me'

In [72]:
df['departure_time'] = pd.to_datetime(df['departure_time'])
df['trip_direction_text'] = df['trip_direction'].map(lambda x: get_trip_direction_text(x))

df_to_gf = df[df['trip_direction'] == 0]
df_to_me = df[df['trip_direction'] == 1]
agg_df = df.groupby(by='trip_direction_text')['trip_duration'].agg(['mean','median','std'])

trips = [df_to_gf, df_to_me]

In [105]:
fig = go.Figure()

for trip in trips:
    
    fig.add_trace(go.Box(
        y = trip.trip_duration,
        name = trip.trip_direction_text.values[0]
    )
    )

fig.update_layout(
    title = 'Transit Trip Duration',
    yaxis_title = 'Minutes',
    template = 'plotly_white',
    showlegend = False
)
fig.show()

In [109]:
fig = go.Figure()

for trip in trips:

    fig.add_trace(go.Scatter(
        x = trip.departure_time,
        y = trip.trip_duration,
        name = trip['trip_direction_text'].values[0],
#         mode = 'markers'
    ))
    
fig.update_layout(
    title = 'Transit Trip Duration',
    yaxis_title = 'Minutes',
    template = 'plotly_white'
)

fig.show()

In [13]:
cursor.close()
cnx.close()

In [151]:
agg_df

,mean,median,std
trip_direction_text,,,
Trip to gf,59.414141,60,4.985714
Trip to me,54.828283,54,3.313671


In [154]:
fig = go.Figure()

for index in agg_df.index:
    
    fig.add_trace(go.Bar(
        x = agg_df.columns,
        y = agg_df.loc[index],
        name = index
    ))
    
fig.update_layout(
    title = 'Descriptive Statistics',
    template = 'plotly_white'
)
    
fig.show()
